# Notebook: Explore Addressed Topics

This notebook is used to explore the topics discussed in reviews previously discussed. 

## Packages

In [2]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP
import pandas as pd
import spacy
import nltk
import re

## Constants

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
DATASET_PATH = "reviews_sentences.csv"
STOPWORDS = set(stopwords.words('german'))
N_TOPICS = 11

## Code

In [5]:
#spacy.cli.download("de_core_news_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 42.3 MB/s eta 0:00:00


In [6]:
nlp = spacy.load('de_core_news_sm')

In [7]:
dataset = pd.read_csv(DATASET_PATH)

In [8]:
def clean_text(text):
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)

    text = re.sub(r'@\w+', '', text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)
    return text.strip()

def lemmatize_remove_stopwords_text(text):
    doc = nlp(text)
    lemmatized_text = ' '.join([token.lemma_.lower() for token in doc if token.text.lower() not in STOPWORDS])
    return lemmatized_text

dataset["text"] = dataset["text"].apply(clean_text)
dataset["text"] = dataset["text"].apply(lemmatize_remove_stopwords_text)

In [9]:
dataset = dataset.text.to_list()

In [10]:
# Use default UMAP but add random state
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=43)
model = BERTopic(language="german", nr_topics=N_TOPICS, umap_model=umap_model, top_n_words=20, embedding_model="paraphrase-multilingual-MiniLM-L12-v2")

In [11]:
topics, probabilities = model.fit_transform(dataset)
topics = model.reduce_outliers(dataset, topics, strategy="distributions")
model.update_topics(dataset, topics, top_n_words=20)

In [12]:
topic_statistics = model.get_document_info(dataset)["Topic"].value_counts().reset_index().rename(columns={"index": "Topic", "Topic": "Frequency"}).sort_values("Topic").reset_index(drop=True)
topic_statistics = topic_statistics[topic_statistics["Topic"] != -1]
topic_statistics["Topic"] = topic_statistics["Topic"] + 1
topic_statistics

,Topic,Frequency
1,1,2043
2,2,1282
3,3,310
4,4,80
5,5,73
6,6,72
7,7,36
8,8,26
9,9,25
10,10,11


In [13]:
top_words = []
for topic in topic_statistics["Topic"]:
    words = model.get_topic(topic - 1)[:5]
    top_words.append(', '.join([word[0] for word in words]))
    
topic_statistics["Top Words"] = top_words

def format_frequency(frequency):
    return "{:,}".format(frequency)

# Anwendung der Funktion auf die Spalte "Frequency"
topic_statistics["Frequency"] = topic_statistics["Frequency"].apply(format_frequency)

topic_statistics

,Topic,Frequency,Top Words
1,1,"2,043","restaurant, tisch, loc, kommen, freundlich"
2,2,"1,282","pizza, essen, gut, pizze, salat"
3,3,310,"preis, bezahlen, leistung, geld, trinkgeld"
4,4,80,"atmosphäre, kalt, ambiente, toll, angenehm"
5,5,73,"toilette, hund, sauber, boden, dreckig"
6,6,72,"personal, arrogant, unfreundlich, burger, mita..."
7,7,36,"schade, katastrophe, katastrophal, schaden, eugen"
8,8,26,"blockhouse, block, house, seit, jahr"
9,9,25,"musik, laut, lautstärke, stimmung, angenehm"
10,10,11,"gericht, getränk, zweiter, erster, pflegen"
